In [349]:
import pandas as pd
import numpy as np
import re
from langdetect import detect
from sklearn.model_selection import train_test_split
import cld3
import math

In [327]:
train = pd.read_csv("data/train.csv")
train.head()

,Artist,Song,Genre,Language,Lyrics
0,12 stones,world so cold,Rock,en,"It starts with pain, followed by hate\nFueled ..."
1,12 stones,broken,Rock,en,Freedom!\nAlone again again alone\nPatiently w...
2,12 stones,3 leaf loser,Rock,en,"Biting the hand that feeds you, lying to the v..."
3,12 stones,anthem for the underdog,Rock,en,You say you know just who I am\nBut you can't ...
4,12 stones,adrenaline,Rock,en,My heart is beating faster can't control these...


In [315]:
data = pd.read_csv("data/train.csv")
data = data[data['Language']=='en']
data = data[data['Lyrics'].apply(lambda x: '\n' in x)]
data = data.drop_duplicates(subset=['Artist', 'Song', 'Genre'], ignore_index=True)
data = data[data['Lyrics'].apply(lambda x: '---------' not in x)]
data['Lyrics'] = data['Lyrics'].apply(lambda x: re.sub('\[.*\]', ' ', x))

In [316]:
def check_en(line):
    result = cld3.get_language(line)
    lang = result[0]
    reliable = result[2]
    if reliable and lang == 'en':
        return 'en'
    else:
        return 'other'

In [328]:
len(data)

192123

In [317]:
data['reliable_lang'] = data['Lyrics'].apply(check_en)
data = data[data['reliable_lang']=='en']

In [321]:
indices = np.arange(len(data))
train_index, test_index = train_test_split(indices, test_size = 0.1)
train_index, dev_index = train_test_split(train_index, test_size=0.1)

train_data = data.iloc[train_index].reset_index(drop=True)
dev_data = data.iloc[dev_index].reset_index(drop=True)
test_data = data.iloc[test_index].reset_index(drop=True)

In [322]:
train_data.to_csv('data/csv/train.csv', index=False)
dev_data.to_csv('data/csv/dev.csv', index=False)
test_data.to_csv('data/csv/test.csv', index=False)

In [294]:
data['Genre'].value_counts()

Rock          96376
Pop           70438
Metal         15910
Jazz          11701
Folk           6922
Indie          5592
R&B            1572
Electronic      447
Name: Genre, dtype: int64

In [288]:
df = pd.read_csv('data/csv/test.csv')

In [290]:
df = df[df['Genre']=='Pop']

In [280]:
dev_data['Genre'].value_counts()

Rock          8679
Pop           6313
Metal         1490
Jazz          1035
Folk           603
Indie          497
R&B            171
Electronic      36
Name: Genre, dtype: int64

In [184]:
jazz = data[data['Genre']=='Jazz'].reset_index(drop=True)
indices = np.arange(len(jazz))
train_index, test_index = train_test_split(indices, test_size = 0.1)
train_index, dev_index = train_test_split(train_index, test_size=0.1)

In [242]:
path = 'data/train/jazz_train.txt'
with open(path,'w', encoding='utf-8', errors='ignore') as f:
    for i in train_index:
#         f.write("ARTIST NAME: "+jazz['Artist'].iloc[i]+'\n')
#         f.write("GENRE: "+jazz['Genre'].iloc[i]+'\n')
        f.write(jazz['Lyrics'].iloc[i])
        f.write('\n')

In [116]:
for g in list(data['Genre'].value_counts().index):
    df = data[data['Genre']==g]
    artist = np.array(df['Artist']).reshape(-1,1)
    genre = np.array(df['Genre']).reshape(-1,1)
    lyrics = np.array(df['Lyrics']).reshape(-1,1)

    out = np.hstack((artist, genre, lyrics))
    file_path = 'data/'+g+'_train.npy'
    np.save(file_path, out)

In [105]:
artist = np.array(rnb['Artist']).reshape(-1,1)
genre = np.array(rnb['Genre']).reshape(-1,1)
lyrics = np.array(rnb['Lyrics']).reshape(-1,1)

rnb_out = np.hstack((artist, genre, lyrics))
np.save('data/rnb_train.npy', rnb_out)

In [93]:
detect(rnb['Lyrics'].iloc[0])

'en'

In [90]:
rnb['detect_lang'] = rnb['Lyrics'].apply(lambda x: detect(x[:50]))

/Users/haowei/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
data['Genre'].value_counts()

Rock          103721
Pop            83620
Metal          17207
Jazz           11760
Folk            7023
Indie           6033
R&B             1575
Electronic       462
Name: Genre, dtype: int64

In [39]:
print(rnb.iloc[20]['Lyrics'])


 

so many people 
I know only a few 
yes I may say that I love this man 
and that man 
but what keeps me from loving you? 
date of birth  geography 
the color of my skin  ideology 
you got ten fingers  two legs  one nose 
like me 
just like me 
and it's as simple as that 
you see 
and if I don't know who to love 
I love them all 
and if I don't know who to trust 
I trust them all 
and if I don't know who to kill 
I may kill myself instead 
from the mouth of a baby 
will come the world-saving words 
that will save us all 
and from the lungs of a child 
will come the everlasting breath of god 
increasing peace and honesty 
and not carrying on despite of me 
don't you know 
this ain't about no race, no creed 
no race, no creed 
and it's as simple as that 
you see 
and if I don't know who to love 
I love them all 
and if I don't know who to trust 
I trust them all 
and if I don't know who to kill 
no suicide 
I'm already dead 




In [359]:
def create_ngram_model(model_class, path, n=2, k=0, type='csv', genre=None):
    ''' Creates and returns a new n-gram model trained on path file '''
    """
    path: numpy file of dimension nx3, columns: artist, genre, lyrics
    
    return: updated model of model_class
    """
    num_iterations = 100  # number of samples to draw
    length = 20  # how many lines in one sample

    model = model_class(n, k)
    if type == 'txt':
        with open(path, encoding='utf-8', errors='ignore') as f:
            lyrics = f.read()
    else:
        lyrics = csv_to_text(path, genre)

    for i in range(num_iterations):
        temp = lyrics_chunks(lyrics, length)
        model.update(temp)

    return model

def start_pad(n):
    ''' Returns a padding string of length n to append to the front of text
        as a pre-processing step to building n-grams '''
    return '~' * n

def ngrams(n, text):
    ''' Returns the ngrams of the text as tuples where the first element is
        the length-n context and the second is the character '''
    adjusted_txt = start_pad(n) + text
    gramlist = []
    for i in range(n, len(adjusted_txt)):
        ngram = (adjusted_txt[i-n:i], adjusted_txt[i])
        gramlist.append(ngram)
    return gramlist

def csv_to_text(path, genre=None):
    ''' Takes in a csv file and return the lyrics concatenated into one text file'''
    df = pd.read_csv(path)
    if genre:
        df = df[df['Genre']==genre]
    lyrics = '\n'.join(list(df['Lyrics']))
    return lyrics

def lyrics_chunks(lyrics, length):
    """
    randomly samples lyrics chunks of specified length

    :param lyrics: str, entire lyrics corpus
    :param length: int, number of lines in one lyrics chunks
    :return: str, lyric lines joined by new line
    """
    lyrics_lines = lyrics.split('\n')
    start_idx = random.randint(0, len(lyrics_lines)-1)
    end_idx = min(start_idx+length, len(lyrics_lines))
    selected_lyrics = lyrics_lines[start_idx:end_idx]
    return '\n'.join(selected_lyrics)

################################################################################
# Model
################################################################################

class NgramModel(object):
    ''' A basic n-gram model using add-k smoothing '''

    def __init__(self, n, k):
        self.n = n
        self.k = k
        self.vocabs = []
        self.context_list = {}
        self.context_count = {}

    def get_vocab(self):
        ''' Returns the set of characters in the vocab '''
        return set(''.join(self.vocabs))

    def update(self, text):
        ''' Updates the model n-grams based on text '''
        self.vocabs.append(text)
        # self.sorted_vocabs = list(set(self.vocabs))
        # self.sorted_vocabs.sort()
        gramlist = ngrams(self.n, text)
        for pair in gramlist:
            context = pair[0]
            char = pair[1]
            if context in self.context_list: # context previously showed up
                self.context_count[context]+=1
                if char in self.context_list[context]: # combination previously showed up
                    self.context_list[context][char]+=1
                else:
                    self.context_list[context][char]=1
            else:
                self.context_count[context]=1
                self.context_list[context] = {char: 1}

    def prob(self, context, char, lambdas=None):
        ''' Returns the probability of char appearing after context '''
        if context not in self.context_list:
            return 1/len(self.get_vocab())

        # if char not in self.context_list[context]:
        #     return 0.0
        # else:
        #     return self.context_list[context][char]/self.context_count[context]
        count = 0
        if char in self.context_list[context]:
            count = self.context_list[context][char]
        p = (count+self.k)/(self.context_count[context]+self.k*len(self.get_vocab()))
        return p

    def random_char(self, context):
        ''' Returns a random character based on the given context and the
            n-grams learned by this model '''
        vocabs = list(self.get_vocab())
        vocabs.sort()
        total_prob = 0
        r = random.random()
        for i in range(len(vocabs)):
            total_prob += self.prob(context, vocabs[i])
            if total_prob > r:
                return vocabs[i]
        return vocabs[-1]

    def random_text(self, length, context=None):
        ''' Returns text of the specified character length based on the
            n-grams learned by this model '''
        if context == None:
            context = start_pad(self.n)
        else:
            context = context[-self.n:]
        text = ''
        for i in range(length):
            char = self.random_char(context)
            text += char
            context = context[1:]+char
        return text

    def random_lines(self, num_lines, context=None):
        ''' Returns specified number of lines based on learned n-grams model'''
        if context == None:
            context = start_pad(self.n)
        else:
            context = context[-self.n:]
        text = ''
        count = 0
        while True:
            char = self.random_char(context)
            if char == '\n':
                count += 1
                if count == num_lines:
                    break
            text += char
            context = context[1:] + char
        return text
    
    def perplexity(self, text,lambdas=None):
        ''' Returns the perplexity of text based on the n-grams learned by
            this model '''
        padded_text = start_pad(self.n) + text
        context = start_pad(self.n)
        log_pp = 0
        for i in range(self.n, len(padded_text)):
            p = self.prob(context, padded_text[i])
            if p == 0:
                return float('inf')
            log_pp += math.log(p)
            context = context[1:]+padded_text[i]
        log_pp = log_pp/(-len(text))
        pp = math.exp(log_pp)
        return pp

In [393]:
m = create_ngram_model(NgramModel, 'data/csv/train.csv', 5,0.0000001)

In [391]:
print(m.random_lines(10))

that's all it takes me higher,
My woman keeps me warm.
 
What can, not live yet never though my style two thumbs up like using analogues haha
I wreck shit for you
Our reason to five pieces
like the car, like fuck the very next day 
Then I hold you burn so easily
What I'd give to me
Is it d-d-destiny, d-destiny, d-destiny, d-destiny, d-destiny, d-destiny, d-destiny, d-destiny


In [386]:
def avg_perplexity(model, test_df, model_type='ngram'):
    """
    computes perplexity of unseen text using given model

    :param model: learned model
    :param test_df:
    :param model_type: 'ngram' or 'neural', default is 'ngram'
    :return: average perplexity score from 20 songs
    """
    tot_pp = 0
    num_samp = 500

    if model_type == 'ngram':
        # select 20 random songs from test data and compute perplexity
        for i in range(num_samp):
            idx = random.randint(0,len(test_df)-1)
            lyric = test_df.iloc[idx]['Lyrics']
            tot_pp += model.perplexity(lyric)

#     if model_type == 'neural':
#         for i in range(num_samp):
#             idx = random.randint(len(test_df))
#             lyric = test_df.iloc[idx]['Lyrics']
#             tot_pp += nn_perplexity(lyric, model)

    return tot_pp / num_samp

# def nn_perplexity(val_text, model):
#     input = char_tensor(val_text[:-1])
#     target = char_tensor(val_text[1:])
#     hidden = model.init_hidden()
#     criterion = nn.CrossEntropyLoss()
#     loss = 0
#     for i in range(len(val_text)-1):
#         output, hidden = model(input[i], hidden)
#         loss += criterion(output, target[i].view(-1))
#     avg_loss = loss.item()/(len(val_text)-1)
#     return np.exp(avg_loss)

# def char_tensor(string):
#     tensor = torch.zeros(len(string)).long()
#     for c in range(len(string)):
#         tensor[c] = all_characters.index(string[c])
#     return Variable(tensor)

In [338]:
test_df = pd.read_csv('data/csv/test.csv')

In [351]:
idx = random.randint(0,len(test_df)-1)
lyric = test_df.iloc[idx]['Lyrics']
# tot_pp += model.perplexity(lyric)

In [352]:
lyric

"In the middle of the night,\nNo one heard the boy\nWho cried himself to sleep\nDidn't care about himself 'cause he\nHurt too much, he was hurt too much\nAnd the burden of the world was put\nOn a hurting child, hurting child\nWho never smiled\nThe world was put on the hurting child,\nHurting child\nI sing for the hurting child\nI sing for the hurting child\n\nIn the middle of the fight\nNo one saw the girl who thought it\nWas her fault, didn't know about her pain\nThough it was right there\nNo one said they cared\nAnd the burden of the world was put\nOn a hurting child, hurting child\nWho never smiled\nThe world was put on the hurting child,\nHurting child\nI sing for the hurting child\n\nI sing for the children, dream for the children,\nCry for the children now\nSo I sing for the children, dream for the children,\nSmile for the children now\nSaid the world was put\nOn a hurting child, hurting child\nWho never smiled\nThe world was put on the hurting child,\nHurting child\nI sing for t

In [394]:
avg_perplexity(m, test_df,'ngram')

149.90236795465054

In [167]:
model = NgramModel(5,0)
for i in range(5000):
    model.update(lyrics[i])

In [168]:
print(lyrics[:5000])

Blue Friday come stripping me away
Put me in your arms
When you play that good reggae
I swear not to roam
I'll always come home
To where I can feel this and know
Blue Friday come stripping me away
Put me in your arms
When you play that good reggae
I swear not to roam
I'll always come home
To where I can feel this and know
The blue night has come
The moon is sending good vibes to everyone
And we all gather around
And head out to our own special place for fun
'Cause here now it seem
It's like a little beach reggae party
Far away from city lights
But blessed by the bloom of the starry nights
Blue Friday come stripping me away
Put me in your arms
When you play that good reggae
I swear not to roam
I'll always come home
To where I can feel this and know
Blue Friday come stripping me away
Put me in your arms
When you play that good reggae
I swear not to roam
I'll always come home
To where I can feel this and know
Sexta blue, sexta blue
Sexta blue, sexta blue
Stomp the ground, yeah
In the tran

In [169]:
model.context_list.keys()

dict_keys(['~~~~~'])

In [170]:
print(model.random_text(100))

aIxPtb[KYIaP
F.dAFzEkN[FSTVDlnnoE,IfqlshVHmz]Js-pbPCWDPxcK-SYyIWL1JqUyrEyd
H1DOWS1c'mf:s
x'jygrbvrez
